In [1]:
!pip install beautifulsoup4 openpyxl pandas

from IPython.display import clear_output
clear_output()

In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd

#change those for desired results
#from when to start gathering bonds
START_DATE = datetime(2026, 2, 20)
#when to finish gathering
END_DATE = datetime.today()
#bond fixing type
FIXING_TYPE = 2

# Funkcja do pobierania danych z tabeli na stronie Bondspot.pl
def get_bondspot_data(start_d, end_d, fix):
    start_date = start_d
    end_date = end_d
    print(start_date, end_date)
    all_data = []
    
    current_date = start_date
    while current_date <= end_date:
        formatted_date = current_date.strftime("%Y%m%d")
        year = current_date.year
        month = current_date.strftime("%m")
        
        url = f'https://www.bondspot.pl/fixing_obligacji?date={formatted_date}&month={month}&year={year}&type={str(fix)}'
        response = requests.get(url)
        
        if response.status_code == 200:
            print(f'Response correct for {formatted_date}')
            soup = BeautifulSoup(response.text, 'html.parser')
            tables = soup.find_all('table', {'class': 'tab_01'})
            if len(tables) > 1:
                table = tables[1]
                
                headers = [th.text.strip() for th in table.find('thead').find_all('th')]
                
                rows = table.find('tbody').find_all('tr')
                for row in rows:
                    columns = row.find_all('td')
                    if len(columns) > 0:
                        data = {headers[i]: columns[i].text.strip() for i in range(len(columns))}

                        data['Date'] = current_date.strftime("%Y-%m-%d")
                        
                        all_data.append(data)
        
        current_date += timedelta(days=1)
    
    df = pd.DataFrame(all_data)
    
    return df


df_bonds = get_bondspot_data(START_DATE, END_DATE, FIXING_TYPE)
print(df_bonds.head())
df_bonds.to_csv('./data/bondspot_data_2026.csv', index=False)


2026-02-20 00:00:00 2026-02-25 11:01:28.406202
Response correct for 20260220
Response correct for 20260221
Response correct for 20260222
Response correct for 20260223
Response correct for 20260224
Response correct for 20260225
  LP   Nazwa          ISIN  Cena K  Cena S Rent.K Rent.S Cena fix Rent fix  \
0  1  DS0726  PL0000108866   99.64   99.85   3.33   2.82    99.75     3.07   
1  2  PS1026  PL0000113460   98.04   98.25   3.25   2.93    98.15     3.08   
2  3  WZ1126  PL0000113130  100.50  100.59      -      -   100.55        -   
3  4  OK0127  PL0000117289   97.15   97.37   3.20   2.94    97.26     3.07   
4  5  PS0527  PL0000114393  100.57  100.76   3.27   3.11   100.67     3.18   

         Date  
0  2026-02-20  
1  2026-02-20  
2  2026-02-20  
3  2026-02-20  
4  2026-02-20  
